# Bitcoin Address Concepts

## Generate the bitcoin address

### Step: 1. SHA-256

In [1]:
import ecdsa
import hashlib
import binascii

def sha256_hash_public_key(public_key: ecdsa.VerifyingKey) -> str:
    return binascii.hexlify(hashlib.sha256(public_key.to_string("uncompressed")).digest()).decode("utf-8")

loaded_public_key: ecdsa.VerifyingKey = load_public_key(pre_public_key)
print("SHA256 hash from public key: ", sha256_hash_public_key(loaded_public_key))

# Generate a bitcoin address from a private key
def sha256_hash_private_key(private_key: ecdsa.SigningKey) -> str:
    return sha256_hash_public_key(generate_public_key(private_key))

loaded_private_key = load_private_key(pre_private_key)
sha256_hash = sha256_hash_private_key(loaded_private_key)
print("SHA256 hash from private key: ", sha256_hash)

NameError: name 'ecdsa' is not defined

### Step 2. RIPEMD-160

In [ ]:
# Find the ripemd160 hash
def ripemd_160_hash(sha256_hash: str) -> str:
    return hashlib.new("ripemd160", bytes.fromhex(sha256_hash)).hexdigest()

ripemd_160_hash_ = ripemd_160_hash(sha256_hash)
print("Ripemd160 hash: ", ripemd_160_hash_)

Ripemd160 hash:  5acee0cdc3f682ef1aaf5d0b387f32270ac4e408


### Step 3. Prefix with version byte

0x00 for Main Network

In [ ]:
 # Add the version byte
def add_version_byte(ripemd_160_hash: str) -> str:
    return "00" + ripemd_160_hash

version_byte = add_version_byte(ripemd_160_hash_)
print("Version byte: ", version_byte)

Version byte:  005acee0cdc3f682ef1aaf5d0b387f32270ac4e408


### Step 4. Append Checksum onto the Extended RIPEMD-160 Result

This step produces the abinary bitcoin address

In [ ]:
# Generate a checksum
def generate_checksum(version_byte: str) -> str:
    return hashlib.sha256(hashlib.sha256(bytes.fromhex(version_byte)).digest()).hexdigest()[:8]

checksum = generate_checksum(version_byte)
print("Checksum: ", checksum)

# Add the checksum to the version byte
def add_checksum(version_byte: str, checksum: str) -> str:
    return version_byte + checksum

checksum_version_byte = add_checksum(version_byte, checksum)
print("Checksum version byte: ", checksum_version_byte)

Checksum:  b21b1f86
Checksum version byte:  005acee0cdc3f682ef1aaf5d0b387f32270ac4e408b21b1f86


### Step 5. Base58Check the binary address

In [ ]:
import base58

# Convert the checksum version byte to base58
def convert_to_base58(checksum_version_byte: str) -> str:
    alphabet = "123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz"
    base58_string = ""
    value = int(checksum_version_byte, 16)
    while value > 0:
        value, mod = divmod(value, 58)
        base58_string = alphabet[mod] + base58_string
    return '1' + base58_string

def convert_to_base58_with_lib(checksum_version_byte: str) -> str:
    return base58.b58encode(bytes.fromhex(checksum_version_byte)).decode("utf-8")

base58_string = convert_to_base58(checksum_version_byte)
print("Base58 string: ", base58_string)

base58_string_with_lib = convert_to_base58_with_lib(checksum_version_byte)
print("Base58 string with lib: ", base58_string_with_lib)

Base58 string:  19H9eQmquVivoCEyVBznuHbYKMcBKdyJkd
Base58 string with lib:  19H9eQmquVivoCEyVBznuHbYKMcBKdyJkd


### Step 6. combine the steps together

In [ ]:
def generate_bitcoin_address(private_key: ecdsa.SigningKey) -> str:
    sha256_hash = sha256_hash_private_key(private_key)
    ripemd_160_hash_ = ripemd_160_hash(sha256_hash)
    version_byte = add_version_byte(ripemd_160_hash_)
    checksum = generate_checksum(version_byte)
    checksum_version_byte = add_checksum(version_byte, checksum)
    base58_string = convert_to_base58(checksum_version_byte)
    return base58_string

loaded_private_key = load_private_key(pre_private_key)
bitcoin_address = generate_bitcoin_address(loaded_private_key)
print("Bitcoin address: ", bitcoin_address)

Bitcoin address:  19H9eQmquVivoCEyVBznuHbYKMcBKdyJkd


In [ ]:
# Generate a bitcoin address from a public key
def generate_bitcoin_address_from_public_key(public_key: ecdsa.VerifyingKey) -> str:
    sha256_hash = sha256_hash_public_key(public_key)
    ripemd_160_hash_ = ripemd_160_hash(sha256_hash)
    version_byte = add_version_byte(ripemd_160_hash_)
    checksum = generate_checksum(version_byte)
    checksum_version_byte = add_checksum(version_byte, checksum)
    base58_string = convert_to_base58(checksum_version_byte)
    return base58_string

loaded_public_key = load_public_key(pre_public_key)
bitcoin_address = generate_bitcoin_address_from_public_key(loaded_public_key)
print("Bitcoin address from public key: ", bitcoin_address)

Bitcoin address from public key:  19H9eQmquVivoCEyVBznuHbYKMcBKdyJkd
